In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

C:\Users\1227458\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:10: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  "LightFM was compiled without OpenMP support. "


In [2]:
from lightfm.evaluation import precision_at_k, recall_at_k

from metrics import precision_at_k as custom_precision, recall_at_k
from utils import prefilter_items

In [391]:
data = pd.read_csv('./data/retail_train.csv')

item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [392]:
sal_val = data_test.groupby('user_id')['sales_value'].mean().reset_index()
sal_val.set_index('user_id', inplace=True)
sal_val['sales_value'].describe()

count    2042.000000
mean        3.563179
std         2.777046
min         0.600000
25%         2.468394
50%         3.022888
75%         3.805497
max        42.990000
Name: sales_value, dtype: float64

In [393]:
sal_val['cheap'] = (sal_val['sales_value'] < 1) * 1
sal_val['middle'] = ((sal_val['sales_value'] >= 1) & (sal_val['sales_value'] < 2)) * 1
sal_val['expensive'] = ((sal_val['sales_value'] >= 2) & (sal_val['sales_value'] < 3)) * 1
sal_val['very_expensive'] = (sal_val['sales_value'] > 3) * 1
sal_val

,sales_value,cheap,middle,expensive,very_expensive
user_id,,,,,
1,2.440488,0,0,1,0
3,2.069474,0,0,1,0
6,3.010781,0,0,0,1
7,2.653130,0,0,1,0
8,2.464186,0,0,1,0
...,...,...,...,...,...
2496,10.000000,0,0,0,1
2497,4.126728,0,0,0,1
2498,2.923500,0,0,1,0


In [394]:
sal_val.drop('sales_value', axis=1, inplace=True)
sal_val

,cheap,middle,expensive,very_expensive
user_id,,,,
1,0,0,1,0
3,0,0,1,0
6,0,0,0,1
7,0,0,1,0
8,0,0,1,0
...,...,...,...,...
2496,0,0,0,1
2497,0,0,0,1
2498,0,0,1,0


In [395]:
data_traint

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,cheap,middle,expensive,very_expensive
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,0.0,0.0,1.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,0.0,0.0,1.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,0.0,0.0,1.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,0.0,0.0,1.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2278485,222,41297772783,635,1120741,1,0.59,304,0.00,1716,91,0.0,0.0,0.0,0.0,1.0,0.0
2278486,462,41297773713,635,993339,1,1.99,304,0.00,2040,91,0.0,0.0,0.0,0.0,1.0,0.0
2278487,462,41297773713,635,995242,1,1.00,304,-0.89,2040,91,0.0,0.0,0.0,0.0,1.0,0.0
2278488,462,41297773713,635,10180324,1,3.00,304,-0.29,2040,91,0.0,0.0,0.0,0.0,1.0,0.0


In [396]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [397]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

C:\Users\1227458\Documents\Учеба\Data Science\Курсы\Рек_системы\lesson_5\webinar_5\utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


In [398]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [399]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [400]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [401]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [402]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


In [403]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [404]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [405]:
user_feat_lightfm = user_feat_lightfm.merge(sal_val, on='user_id', how='left')


In [406]:
user_feat_lightfm.columns

Index(['age_desc_19-24', 'age_desc_25-34', 'age_desc_35-44', 'age_desc_45-54',
       'age_desc_55-64', 'age_desc_65+', 'marital_status_code_A',
       'marital_status_code_B', 'marital_status_code_U',
       'income_desc_100-124K', 'income_desc_125-149K', 'income_desc_15-24K',
       'income_desc_150-174K', 'income_desc_175-199K', 'income_desc_200-249K',
       'income_desc_25-34K', 'income_desc_250K+', 'income_desc_35-49K',
       'income_desc_50-74K', 'income_desc_75-99K', 'income_desc_Under 15K',
       'homeowner_desc_Homeowner', 'homeowner_desc_Probable Owner',
       'homeowner_desc_Probable Renter', 'homeowner_desc_Renter',
       'homeowner_desc_Unknown', 'hh_comp_desc_1 Adult Kids',
       'hh_comp_desc_2 Adults Kids', 'hh_comp_desc_2 Adults No Kids',
       'hh_comp_desc_Single Female', 'hh_comp_desc_Single Male',
       'hh_comp_desc_Unknown', 'household_size_desc_1',
       'household_size_desc_2', 'household_size_desc_3',
       'household_size_desc_4', 'household_size_de

In [407]:
user_feat_lightfm

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown,cheap,middle,expensive,very_expensive
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,1,0.0,0.0,1.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,1.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0.0,0.0,0.0,1.0
2497,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0.0,0.0,0.0,1.0
2498,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0.0,0.0,1.0,0.0


In [409]:
user_feat_lightfm['sales_value_None'] = 0
user_feat_lightfm.loc[user_feat_lightfm['cheap'].isna(), 'sales_value_None'] = 1

In [411]:
user_feat_lightfm.loc[user_feat_lightfm['cheap'].isna(), 'cheap'] = 0
user_feat_lightfm.loc[user_feat_lightfm['middle'].isna(), 'middle'] = 0
user_feat_lightfm.loc[user_feat_lightfm['expensive'].isna(), 'expensive'] = 0
user_feat_lightfm.loc[user_feat_lightfm['very_expensive'].isna(), 'very_expensive'] = 0

In [413]:
user_feat_lightfm

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown,cheap,middle,expensive,very_expensive,sales_value_None
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0.0,0.0,1.0,0.0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0.0,1.0,0.0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0,0,0,1,0,0,1,0,0,0,...,0,1,0,0,0,0.0,0.0,0.0,1.0,0
2497,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0.0,0.0,0.0,1.0,0
2498,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0.0,0.0,1.0,0.0,0


In [567]:
model = LightFM(no_components=40,
                loss='bpr', # "logistic","bpr"
                learning_rate=0.01, 
                item_alpha=0.4,
                user_alpha=0.1, 
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)

In [568]:
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True) 

Epoch: 100%|██████████| 20/20 [05:45<00:00, 17.25s/it]


In [569]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [570]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [571]:
# мы можем использовать встроенные метрики lightFM
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

print(f"Train precision {train_precision}")

Train precision 0.4385262429714203


In [572]:
# подготавливаемм id для юзеров и товаров в порядке пар user-item
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [573]:
# модель возвращает меру/скор похожести между соответствующим пользователем и товаром
predictions = model.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=10)

In [574]:
data_train_filtered['score'] = predictions

In [575]:
predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
            unique().reset_index()

In [576]:
df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')

In [577]:
df_result_for_metrics.head()

,user_id,actual,item_id
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1029743, 6034857, 952163, 1004906, 877391, 85..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 983584, 5585510, 899624, 866211, 946..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1070820, 1029743, 1126899, 1121393, 9524291, ..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1029743, 1126899, 1106523, 1072494, 7147142, ..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1106523, 1070820, 1029743, 6034857, 5585510, ..."


In [578]:
precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
print(f"Precision: {precision}")

Precision: 0.14505747126436602


Я провела подбор параметров, но результат не стал лучше, поэтому оставила прежние, что изначально были в модели. Чуть-чуть улучшить показатель удалось через добавление дополнительного признака: sales_value. Я сделала разбивку на  'cheap', 'middle', 'expensive', 'very_expensive', 'sales_value_None'. 